# Load Python Packages

In [1]:
import joblib
import keras
import tensorflow as tf
import pyfaidx
import math
import pandas as pd
import tqdm
import numpy as np
import deepdish
import pyBigWig
import scipy
from modisco.visualization import viz_sequence
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import logomaker
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

/users/wangtony/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
/users/wangtony/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps, cop

# Load Data 

In [2]:
genome = pyfaidx.Fasta('/users/surag/genomes/mm10/mm10.fa')
seqs = np.load("/mnt/lab_data3/surag/kundajelab/playground/src/analyses/20220427_hit_scoring_NN/out/native_shap/shap.npy")
labels = joblib.load("/mnt/lab_data3/surag/kundajelab/playground/src/analyses/20220427_hit_scoring_NN/out/native_shap/labels.joblib")

In [3]:
encoded_labels = np.zeros((84849,2114,11)) 
counter = 0 
for i in labels: 
    length = len(i)
    for x in range(length): 
        array = i[x]
        motif_class = array[0]
        seq_start = array[1]
        seq_end = array[2]
        for y in range(2114): 
            if y != seq_start: 
                encoded_labels[counter][y][10] = 1
            else: 
                encoded_labels[counter][seq_start][motif_class] = 1 
    counter = counter + 1
print(encoded_labels)

[[[0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]]

 ...

 [[0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]]]


# Shuffle the Dataset and Create 8:1:1 Train Val Test Splits

In [4]:
idxs = np.array(range(seqs.shape[0]))
np.random.shuffle(idxs)
train_idxs = idxs[:int(0.8*len(idxs))]
val_idxs = idxs[int(0.8*len(idxs)):int(0.9*len(idxs))]
test_idxs = idxs[int(0.9*len(idxs)):]

train_seqs = seqs[train_idxs]
train_labels = encoded_labels[train_idxs]

val_seqs = seqs[val_idxs]
val_labels = encoded_labels[val_idxs]

test_seqs = seqs[test_idxs]
test_labels = encoded_labels[test_idxs]

# Define Model 

In [5]:
def model_3conv(channels=50, blocks=8):
    inp = tf.keras.Input((2114,4))
    x = tf.keras.layers.Conv1D(channels, 20, padding='same', activation='relu')(inp)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)

    
    for i in range(blocks):
        x1 = tf.keras.layers.Conv1D(channels, 15, padding='same', activation='relu')(x)
        x1 = tf.keras.layers.BatchNormalization()(x1)
        x1 = tf.keras.layers.Dropout(0.2)(x1)
        x = tf.keras.layers.Add()([x,x1])
    
    x = tf.keras.layers.Dense(11)(x)
    

    return tf.keras.Model(inputs=inp, outputs=x)

In [6]:
model = model_3conv()

In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2114, 4)]    0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 2114, 50)     4050        ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 2114, 50)    200         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 dropout (Dropout)              (None, 2114, 50)     0           ['batch_normalization[0][0]']

                                                                                                  
 batch_normalization_8 (BatchNo  (None, 2114, 50)    200         ['conv1d_8[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 dropout_8 (Dropout)            (None, 2114, 50)     0           ['batch_normalization_8[0][0]']  
                                                                                                  
 add_7 (Add)                    (None, 2114, 50)     0           ['add_6[0][0]',                  
                                                                  'dropout_8[0][0]']              
                                                                                                  
 dense (Dense)                  (None, 2114, 11)     561         ['add_7[0][0]']                  
          

# Downweight the Loss for Background

In [8]:
def motif_loss_wrapper(background_weight=0.1,background_labels_index=-1):
    print("background_weight: ",background_weight)
    def motif_loss(y_true,logits):
        eps=1e-5
        y_pred = tf.nn.softmax(logits, axis=-1)
        num_classes = y_pred.shape[-1]
        cce_loss = -num_classes*(y_true*tf.math.log(y_pred+eps))
        weight_vector = [1] * num_classes
        weight_vector[background_labels_index] = background_weight
        weight_vector = np.array(weight_vector)
        weighted_cce_loss = cce_loss * weight_vector
        return tf.math.reduce_mean(weighted_cce_loss) * 100
    return motif_loss

# Train Model

In [9]:
model = model_3conv()
# cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
def acc_within_nonbg(y_true, y_pred):
    return tf.reduce_mean(tf.cast(tf.equal(tf.math.argmax(y_true, -1), tf.math.argmax(y_pred, -1))[tf.math.argmax(y_true, -1) != 10],
                                  tf.float32))

model.compile(
    optimizer="adam",
    loss=motif_loss_wrapper(),
    metrics=[tf.keras.metrics.CategoricalAccuracy(), acc_within_nonbg],
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_acc_within_nonbg', factor=0.2,
                              patience=5, min_lr=0.0001)
model.fit(train_seqs,
          train_labels,
         batch_size=128,
         epochs=50,
         validation_data=(val_seqs, val_labels),
         callbacks=reduce_lr,
         shuffle=True)

background_weight:  0.1
Epoch 1/50
531/531 [==============================] - 209s 382ms/step - loss: 1.2354 - categorical_accuracy: 0.9884 - acc_within_nonbg: 0.5043 - val_loss: 0.2217 - val_categorical_accuracy: 0.9995 - val_acc_within_nonbg: 0.1700 - lr: 0.0010
Epoch 2/50
531/531 [==============================] - 201s 379ms/step - loss: 0.0933 - categorical_accuracy: 0.9991 - acc_within_nonbg: 0.8136 - val_loss: 0.2994 - val_categorical_accuracy: 0.9972 - val_acc_within_nonbg: 0.9888 - lr: 0.0010
Epoch 3/50
531/531 [==============================] - 199s 376ms/step - loss: 0.0605 - categorical_accuracy: 0.9994 - acc_within_nonbg: 0.8666 - val_loss: 0.0403 - val_categorical_accuracy: 0.9994 - val_acc_within_nonbg: 0.9663 - lr: 0.0010
Epoch 4/50
531/531 [==============================] - 199s 375ms/step - loss: 0.0512 - categorical_accuracy: 0.9994 - acc_within_nonbg: 0.8848 - val_loss: 0.0572 - val_categorical_accuracy: 0.9990 - val_acc_within_nonbg: 0.9831 - lr: 0.0010
Epoch 5/50
5

531/531 [==============================] - 199s 374ms/step - loss: 0.0266 - categorical_accuracy: 0.9996 - acc_within_nonbg: 0.9472 - val_loss: 0.0242 - val_categorical_accuracy: 0.9996 - val_acc_within_nonbg: 0.9815 - lr: 1.0000e-04
Epoch 35/50
531/531 [==============================] - 199s 374ms/step - loss: 0.0261 - categorical_accuracy: 0.9996 - acc_within_nonbg: 0.9489 - val_loss: 0.0238 - val_categorical_accuracy: 0.9996 - val_acc_within_nonbg: 0.9795 - lr: 1.0000e-04
Epoch 36/50
531/531 [==============================] - 198s 374ms/step - loss: 0.0259 - categorical_accuracy: 0.9997 - acc_within_nonbg: 0.9496 - val_loss: 0.0243 - val_categorical_accuracy: 0.9996 - val_acc_within_nonbg: 0.9835 - lr: 1.0000e-04
Epoch 37/50
531/531 [==============================] - 198s 374ms/step - loss: 0.0257 - categorical_accuracy: 0.9997 - acc_within_nonbg: 0.9511 - val_loss: 0.0229 - val_categorical_accuracy: 0.9996 - val_acc_within_nonbg: 0.9798 - lr: 1.0000e-04
Epoch 38/50
531/531 [=======

In [10]:
model.save("new1_motif_model.hdf5")
# from keras.models import load_model
# regularized_keras_model=load_model("new_motif_model.hdf5", custom_objects={"acc_within_nonbg": acc_within_nonbg})

In [12]:
def acc_within_nonbg(y_true, y_pred):
    return tf.reduce_mean(tf.cast(tf.equal(tf.math.argmax(y_true, -1), tf.math.argmax(y_pred, -1))[tf.math.argmax(y_true, -1) != 10],
                                  tf.float32))
from keras.models import load_model
model2=load_model("new1_motif_model.hdf5", custom_objects={"acc_within_nonbg": acc_within_nonbg})

ValueError: Unknown loss function: motif_loss. Please ensure this object is passed to the `custom_objects` argument. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

# Calculate Accuracy on Test Set and Visualize Confusion Matrix 

In [ ]:
test_pred = model.predict(test_seqs)
acc_within_nonbg(test_labels, test_pred).numpy()
conf_mat = confusion_matrix((np.argmax(test_labels, -1)[np.argmax(test_labels, -1)!=10]).flatten(), 
                            (np.argmax(test_pred, -1)[np.argmax(test_labels, -1)!=10]).flatten())

ax = sns.heatmap(conf_mat, annot=False, cmap='Blues')

# Visualize Prediction on Test Set 

In [ ]:
plt.rcParams["figure.figsize"] = (20,3)
IDX=800
plt.plot(np.argmax(test_labels[IDX], -1)[np.argmax(test_labels[IDX], -1)!=10], label='label')
plt.plot(np.argmax(test_pred[IDX], -1)[np.argmax(test_labels[IDX], -1)!=10], label='pred')
plt.legend()
plt.show()

plt.plot(np.argmax(test_labels[IDX], -1), label='label')
plt.plot(np.argmax(test_pred[IDX], -1), label='pred')
plt.legend()
plt.show()

In [ ]:
IDX=800
fig, ax = plt.subplots(1, figsize=(20,2))
logomaker.Logo(pd.DataFrame(test_seqs[IDX][600:1200], columns=['A','C','G','T']), ax=ax)
plt.plot(np.argmax(test_labels[IDX][600:1200], -1)/100, label='label')
plt.plot(np.argmax(test_pred[IDX][600:1200], -1)/100, label='pred')
ax.set_ylim(0,.11)
plt.legend()
plt.show()

# Visualize Prediction on Real Sequences 

In [ ]:
import os
HDF5_PATH = "/oak/stanford/groups/akundaje/projects/cad/outs/10_5_2021_bpnet_training_valid_arch/full_models/c0/fold0.counts_scores.h5"
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
imp_scores = deepdish.io.load(HDF5_PATH)
shap_abssum = np.abs(imp_scores['projected_shap']['seq'][:,:,imp_scores['projected_shap']['seq'].shape[1]//2-250:imp_scores['projected_shap']['seq'].shape[1]//2+250]).sum(1).sum(-1)
top = np.random.choice(np.argsort(shap_abssum)[::-1][:20000], 100)
top_seq = imp_scores['projected_shap']['seq'][top].transpose(0,2,1)
top_pred = model.predict(top_seq)

In [ ]:
IDX=3

sns.heatmap(top_pred[IDX][900:1200].T)
plt.show()

fig, ax = plt.subplots(1, figsize=(40,2))
logomaker.Logo(pd.DataFrame(top_seq[IDX][900:1200], columns=['A','C','G','T']), ax=ax)
plt.plot(np.argmax(top_pred[IDX][900:1200,], -1)/100, label='pred')
ax.set_ylim(0,.11)
plt.legend()
plt.show()

In [ ]:
!ls /oak/stanford/groups/akundaje/projects/cad/outs/10_5_2021_bpnet_training_valid_arch/